#### In this notebook we will extract annotations from SeqVec embeddings via trained models that can predict secondary structure and subcellular localization

In [1]:
from bio_embeddings.embed import SeqVecEmbedder

We initialize the SeqVec embedder by providing the model files, which were [downloaded from here](http://maintenance.dallago.us/public/embeddings/embedding_models/seqvec/) previously.

In [2]:
embedder = SeqVecEmbedder(weights_file="/mnt/nfs/models/seqvec/weights_file", options_file="/mnt/nfs/models/seqvec/options_file")

We select an AA sequence. In this case, the sequence is that of [Aspartate aminotransferase, mitochondrial](https://www.uniprot.org/uniprot/P12345)


In [3]:
target_sequence = "MALLHSARVLSGVASAFHPGLAAAASARASSWWAHVEMGPPDPILGVTEAYKRDTNSKKMNLGVGAYRDDNGKPYVLPSVRKAEAQIAAKGLDKEYLPIGGLAEFCRASAELALGENSEVVKSGRFVTVQTISGTGALRIGASFLQRFFKFSRDVFLPKPSWGNHTPIFRDAGMQLQSYRYYDPKTCGFDFTGALEDISKIPEQSVLLLHACAHNPTGVDPRPEQWKEIATVVKKRNLFAFFDMAYQGFASGDGDKDAWAVRHFIEQGINVCLCQSYAKNMGLYGERVGAFTVICKDADEAKRVESQLKILIRPMYSNPPIHGARIASTILTSPDLRKQWLQEVKGMADRIIGMRTQLVSNLKKEGSTHSWQHITDQIGMFCFTGLKPEQVERLTKEFSIYMTKDGRISVAGVTSGNVGYLAHAIHQVTK"

We produce the embeddings of the above sequence. Since we only have one sequence, we use the simple `embed` function, rather than the `embed_many` or `embed_batch`, which we would instead use if we had multiple sequences to embed.

In [4]:
embedding = embedder.embed(target_sequence)

The `bio_embeddings` pipeline includes some models trained on embeddings for the prediction of Secondary Structure and Subcellular Localization. In the following we make use of these models.

To speed up processing, we have downloaded the model weights of the supervised subcellular localization and secondary structure prediction models from [here](http://maintenance.dallago.us/public/embeddings/feature_models/seqvec/).

In [5]:
from bio_embeddings.extract.basic import BasicAnnotationExtractor

In [6]:
annotations_extractor = BasicAnnotationExtractor(
    secondary_structure_checkpoint_file="../../models/seqvec/secstruct_checkpoint.pt",
    subcellular_location_checkpoint_file="../../models/seqvec/subcell_checkpoint.pt"
)

In [7]:
annotations = annotations_extractor.get_annotations(embedding)

Let's see what annotations are available from SeqVec

In [8]:
annotations._fields

('DSSP3', 'DSSP8', 'disorder', 'localization', 'membrane')

Let's print the subcellular localization predicted via the SeqVec embeddings

In [9]:
print(f"The subcellular localization predicted from the embedding is: {annotations.localization.value}")

The subcellular localization predicted from the embedding is: Mitochondrion


For AA-annotations, e.g. secondary structure, we can use a helper function to format the extracted annotations as a single string:

In [10]:
from bio_embeddings.utilities.helpers import convert_list_of_enum_to_string

In [11]:
print("The predicted secondary structure (red) of the sequence is:")

for (AA, DSSP3) in zip(target_sequence, convert_list_of_enum_to_string(annotations.DSSP3)):
    print(f"\x1B[30m{AA}\x1b[31m{DSSP3}")

The predicted secondary structure (red) of the sequence is:
MC
AC
LC
LC
HC
SC
AC
RC
VC
LC
SC
GC
VC
AC
SC
AC
FC
HC
PC
GC
LC
AC
AC
AC
AC
SC
AC
RC
AC
SC
SC
WH
WH
AH
HH
VC
EC
MC
GC
PC
PC
DC
PC
IH
LH
GH
VH
TH
EH
AH
YH
KH
RH
DC
TC
NC
SC
KC
KC
ME
NE
LC
GC
VC
GC
AC
YE
RC
DC
DC
NC
GC
KC
PE
YC
VC
LC
PH
SH
VH
RH
KH
AH
EH
AH
QH
IH
AH
AH
KC
GC
LC
DC
KC
EC
YC
LC
PC
IC
GC
GC
LC
AH
EH
FH
CH
RH
AH
SH
AH
EH
LH
AH
LH
GC
EC
NC
SC
EH
VH
VC
KC
SC
GC
RC
FE
VE
TE
VE
QE
TE
IC
SC
GH
TH
GH
AH
LH
RH
IH
GH
AH
SH
FH
LH
QH
RH
FH
FH
KC
FC
SC
RC
DE
VE
FE
LE
PE
KC
PC
SC
WC
GC
NC
HH
TH
PH
IH
FH
RH
DH
AH
GC
MC
QE
LE
QE
SE
YE
RE
YE
YC
DC
PC
KC
TC
CC
GC
FE
DC
FH
TH
GH
AH
LH
EH
DH
IH
SH
KH
IC
PC
EC
QC
SC
VE
LE
LE
LE
HE
AE
CC
AC
HC
NC
PC
TC
GC
VC
DC
PC
RC
PH
EH
QH
WH
KH
EH
IH
AH
TH
VH
VH
KH
KH
RC
NC
LC
FE
AE
FE
FE
DE
MC
AC
YC
QC
GC
FC
AC
SC
GC
DC
GC
DC
KC
DC
AH
WH
AH
VH
RH
HH
FH
IH
EH
QH
GC
IC
NC
VE
CE
LE
CE
QE
SE
YC
AC
KC
NC
MC
GC
LC
YC
GC
EC
RC
VC
GC
AE
FE
TE
VE
IE
CE
KC
DC
AH
DH
EH
AH
KH
RH
VH
EH
SH
QH
LH
KH
IH
LH
IC
RC
P